<a href="https://colab.research.google.com/github/evillag/uncertainty_gan/blob/main/%5BFeature_Densities%5D_CERN_uncertainties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

1.   Download Rich detector dataset (from GDrive)
2.   Download [Uncertainty code](https://gitlab.com/lambda-hse/lhcb-rich-gan-uncertainty.git) from [paper]().
3.   Install dependencies.



In [ ]:
import numpy
!python --version

Python 3.10.12


In [ ]:
IN_COLAB = True

try:
  import google.colab
  from google.colab import drive
  drive.mount('/content/drive')
  !ls '/content/drive/MyDrive/data/rich'
  !git clone https://gitlab.com/lambda-hse/lhcb-rich-gan-uncertainty.git
  !mv lhcb-rich-gan-uncertainty/experiments .
  !mv lhcb-rich-gan-uncertainty/src .
  !rm -r lhcb-rich-gan-uncertainty/
  !rm -r sample_data/
  !pip install tensorflow-addons
except:
  IN_COLAB = False

Mounted at /content/drive
kaon2_+_down_2016_.csv	kaon_+_up_2016_.csv    pion2_+_down_2016_.csv  pion_+_up_2016_.csv
kaon2_-_down_2016_.csv	kaon_-_up_2016_.csv    pion2_-_down_2016_.csv  pion_-_up_2016_.csv
kaon2_+_up_2016_.csv	muon_+_down_2016_.csv  pion2_+_up_2016_.csv    proton_+_down_2016_.csv
kaon2_-_up_2016_.csv	muon_-_down_2016_.csv  pion2_-_up_2016_.csv    proton_-_down_2016_.csv
kaon_+_down_2016_.csv	muon_+_up_2016_.csv    pion_+_down_2016_.csv   proton_+_up_2016_.csv
kaon_-_down_2016_.csv	muon_-_up_2016_.csv    pion_-_down_2016_.csv   proton_-_up_2016_.csv
Cloning into 'lhcb-rich-gan-uncertainty'...
remote: Enumerating objects: 210, done.
remote: Total 210 (delta 0), reused 0 (delta 0), pack-reused 210
Receiving objects: 100% (210/210), 2.94 MiB | 26.89 MiB/s, done.
Resolving deltas: 100% (94/94), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 4.5 MB/s eta 0:00:00


In [ ]:
import datetime
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from experiments.efficiency.uncertainty_model_train import train_model
from experiments.efficiency.uncertainty_models import uncertainty_mlp
from experiments.efficiency.uncertainty_utils import (
    efficiency_bands_with_uncertainty, efficiency_momentum_with_uncertainty)
from experiments.efficiency.utils import (
    efficiency_bands, efficiency_momentum, ensemble_and_ref_model_inference,
    ensemble_and_ref_model_inference_on_bands, tf_to_numpy_dataset,
    threshold_selection)

from src.cramer_gan_trainer import CramerGANTrainer
from src.dataset import CramerGANDataset
from src.datasets.utils_rich import (get_merged_typed_dataset,
                                     parse_dataset_np, parse_example)
from src.models.gans.discriminators.fcn_disc import RICHDiscriminator
from src.models.gans.generators.fcn_gen import RichMCDropFunc, VirtualEnsembleModel

from scipy.spatial import distance
from tqdm import trange
from tqdm import tqdm

print(f'GPU available? {tf.config.list_physical_devices("GPU")}')

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


GPU available? [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# GLOBALS
PARTICLES = ["pion", 'kaon', "muon", "proton"]
# DROPOUTS = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
DROPOUTS = [0.25, 0.3, 0.35, 0.4]
ENSEMBLES = [16, 32, 64, 128, 256]
NUM_REPS = 10
SUB_SAMPLE_SIZE = .3
THRESHOLD = 1.0

DATA_DIR = '/content/drive/MyDrive/data/rich' if IN_COLAB else '../data/rich'
CHECKPOINT_BASE = '/content/drive/MyDrive/Documentos/Maestria ITCR/cern/checkpoints/' if IN_COLAB else '../checkpoints'
CKPT_NUMBER = 'ckpt-21'

def get_checkpoint_name(particle):
  return f'bernoulli_structured_dropout_line_test_cramer_weighted_{particle}'

# Particle selection and dataset split

 Line experiment is mainly focused on testing whether the ensemble-based uncertainty is applicable for out-of-distribution data. To check this, we make train/test split as two disjoint sets. Taking the symmetry of the data after standardization, we split the data by the line $y=x$ (we consider pseudorapidity (ETA) and momentum (P) for split).

 ------
 Notes


*   Data is split based on two dimensions arbitrarily (P abd ETA)
*   Data is scaled to follow a normal distribution



- Features
  - Brunel_P
  - Brunel_ETA
  - nTracks_Brunel

- Targets:
    - RichDLLe
    - RichDLLk
    - RichDLLmu
    - RichDLLp
    - RichDLLbt

- Discarded
  - probe_sWeight

In [ ]:
def _split_by_line(df, slope=1, intercept=0):
  top_half = df[df['Brunel_ETA'] > df['Brunel_P'] * slope + intercept]
  bottom_half = df[df['Brunel_ETA'] <= df['Brunel_P'] * slope + intercept]

  top_half = top_half.reset_index(drop=True)
  bottom_half = bottom_half.reset_index(drop=True)

  return top_half, bottom_half

def split_by_line(df_train, df_test):
  return _split_by_line(df_train)[0], _split_by_line(df_test)[1]

def load_particle_datasets(particle, data_dir=DATA_DIR):
  """ The returned dictionary has this format:
      {
        "<particle_name>": {
          'data_train': data_train,
          'data_val': data_val,
          'scaler': scaler,
          'feats_train': feats_train,
          'targets_train': targets_train,
          'feats_val': feats_val,
          'targets_val': targets_val
        }
      }
  """
  data_train, data_val, scaler = get_merged_typed_dataset(data_dir, particle, dtype=np.float32, log=True,
                                                          sample_fn=split_by_line)
  feats_train, targets_train, _ = parse_dataset_np(data_train)
  feats_val, targets_val, _ = parse_dataset_np(data_val)

  print(f'feats_train shape\t{feats_train.shape}\n'
        f'targets_train shape\t{targets_train.shape}\n'
        f'feats_val shape  \t{feats_val.shape}\n'
        f'targets_val shape\t{targets_val.shape}\n')

  return {
      'data_train': data_train,
      'data_val': data_val,
      'scaler': scaler,
      'feats_train': feats_train,
      'targets_train': targets_train,
      'feats_val': feats_val,
      'targets_val': targets_val
  }

datasets = {particle: load_particle_datasets(particle) for particle in PARTICLES}

Reading and concatenating datasets:
	/content/drive/MyDrive/data/rich/pion_-_down_2016_.csv
	/content/drive/MyDrive/data/rich/pion_+_down_2016_.csv
	/content/drive/MyDrive/data/rich/pion_-_up_2016_.csv
	/content/drive/MyDrive/data/rich/pion2_-_down_2016_.csv
	/content/drive/MyDrive/data/rich/pion2_+_down_2016_.csv
	/content/drive/MyDrive/data/rich/pion2_+_up_2016_.csv
	/content/drive/MyDrive/data/rich/pion2_-_up_2016_.csv
	/content/drive/MyDrive/data/rich/pion_+_up_2016_.csv
splitting to train/val/test
fitting the scaler
scaler train sample size: 2000000
scaler n_quantiles: 100000, time = 1.9535324573516846
scaling train set
scaling test set
converting dtype to <class 'numpy.float32'>
feats_train shape	(947640, 3)
targets_train shape	(947640, 5)
feats_val shape  	(524888, 3)
targets_val shape	(524888, 5)

Reading and concatenating datasets:
	/content/drive/MyDrive/data/rich/kaon_+_down_2016_.csv
	/content/drive/MyDrive/data/rich/kaon_-_up_2016_.csv
	/content/drive/MyDrive/data/rich/kao

# Models creation (restoring from checkpoint)

 This chapter contains the only model definition and restores the model from the checkpoint. It does not contain training/evaluation routines since it was done in another notebook (look LineTestTrain.ipynb notebook).

We use Cramer GAN modification to train generator learn the distribution. [Link](https://arxiv.org/abs/1705.10743) to Cramer GAN for more details. The training configuration:

* Batch size: 1000
* Critic steps: 15
* Generator optimizer: RMSProp with learning rate 0.0002
* iscriminator optimizer: RMSProp with learning rate 0.0002
* Use weights: True
* Number of epochs: 400

In [ ]:
class MonteCarloDroupoutModel:
  def __init__(self, particle, dropout_rate,
               log_dir='log_dir_tmp',
               checkpoint_base=CHECKPOINT_BASE,
               chekpoint_file=CKPT_NUMBER,
               debug=False):

    self.particle = particle
    self.dropout_rate = dropout_rate
    self.log_dir = log_dir

    print(f'Generating model for {particle} with a dropout rate of {dropout_rate}')

    self._gen_config = {
        'drop_rate': dropout_rate,
        # 'dropout_type': 'bernoulli_structured',
        'dropout_type': 'bernoulli',
        # 'drop_kwargs': {'patch_size': 3}
    }

    self._generator = RichMCDropFunc(**self._gen_config)
    self._generator.build((None, 3))
    self._discriminator = RICHDiscriminator()

    self._checkpoint_dir = os.path.join(checkpoint_base, get_checkpoint_name(self.particle))
    self._filename = os.path.join(self._checkpoint_dir, chekpoint_file)

    if debug:
      print("\nGenerator:\n")
      print(self._generator.summary(line_length=96))
      print("\nDiscriminator:\n")
      print(self._discriminator.summary())
      print(f"\nCheckpoint filename: {self._filename}\n")


    # Model was trained with tensorflow 2.10.1, use the legacy optimizer
    self._generator_optimizer = tf.keras.optimizers.legacy.RMSprop(2e-4)
    self._discriminator_optimizer = tf.keras.optimizers.legacy.RMSprop(2e-4)

    self._trainer_config = {
      'generator': self._generator,
      'discriminator': self._discriminator,
      'generator_optimizer': self._generator_optimizer,
      'discriminator_optimizer': self._discriminator_optimizer,
      'checkpoint_dir': self._checkpoint_dir,
      'log_dir': log_dir
    }
    trainer = CramerGANTrainer(**self._trainer_config)
    # Restore pretrained model
    trainer.restore(self._filename)

  def __str__(self):
     return f"{self.particle}_{self.dropout_rate}"

  def get_generator(self) -> VirtualEnsembleModel:
    return self._generator


# Test model creation
#mc_model = MonteCarloDroupoutModel('kaon', 0.1, debug=True)


In [ ]:
mc_model = MonteCarloDroupoutModel('kaon', 0.1, debug=True)
gen1 = mc_model.get_generator()
gen1.single_model_inference_mode()
gen1.summary()

Generating model for kaon with a dropout rate of 0.1
Layer 0
Layer 1
Layer 2
Layer 3
Layer 4

Generator:

Model: "virtual_ensemble_model"
________________________________________________________________________________________________
 Layer (type)                              Output Shape                          Param #        
 Inputs (InputLayer)                       [(None, 3)]                           0              
                                                                                                
 NoiseInjection (NoiseInjection)           (None, 67)                            0              
                                                                                                
 Layer_0/Dense (Dense)                     (None, 128)                           8704           
                                                                                                
 Layer_0/LeakyReLU (LeakyReLU)             (None, 128)                           0    

In [ ]:
input_data = np.random.rand(1, 3)
input_data

array([[0.96711825, 0.44602729, 0.5387772 ]])

In [ ]:
gen1.predict(input_data)

1/1 [==============================] - 2s 2s/step


array([[-0.0440134 ,  0.7154342 , -0.6578866 , -0.76405543,  0.09815606]],
      dtype=float32)

In [ ]:
len(gen1.layers)

18

In [ ]:
from keras.models import Model

XX = gen1.input
# YY = gen1.layers[14].output
YY =  tf.keras.layers.Dense(128)(gen1.get_layer("Layer_1
/Dense").output)
new_model = Model(XX, YY)
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Inputs (InputLayer)         [(None, 3)]               0         
                                                                 
 NoiseInjection (NoiseInjec  (None, 67)                0         
 tion)                                                           
                                                                 
 Layer_0/Dense (Dense)       (None, 128)               8704      
                                                                 
 Layer_0/LeakyReLU (LeakyRe  (None, 128)               0         
 LU)                                                             
                                                                 
 Layer_0/DropoutTrain (Drop  (None, 128)               0         
 outTrain)                                                       
                                                           

In [ ]:
Xresult = new_model.predict(input_data)
# print(Xresult.shape)
# Xresult

TypeError: <tf.Tensor 'virtual_ensemble_model/Layer_0/DropoutTrain/dropout/SelectV2:0' shape=(1, 128) dtype=float32> is out of scope and cannot be used here. Use return values, explicit Python locals or TensorFlow collections to access it.
Please see https://www.tensorflow.org/guide/function#all_outputs_of_a_tffunction_must_be_return_values for more information.

<tf.Tensor 'virtual_ensemble_model/Layer_0/DropoutTrain/dropout/SelectV2:0' shape=(1, 128) dtype=float32> was defined here:
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start
    File "/usr/local/lib/python3.10/dist-packages/tornado/platform/a

In [ ]:
input_data

array([[0.96711825, 0.44602729, 0.5387772 ]])


/-*-/*-/*-/-*/-*/-*/-*/


The error:

```
TypeError: <tf.Tensor 'model_10/Layer_0/DropoutTrain/dropout/SelectV2:0' shape=(1, 128) dtype=float32> is out of scope and cannot be used here. Use return values, explicit Python locals or TensorFlow collections to access it.
```


Is due to the fact that TensorFlow's tf.function creates a new computational graph for the function it decorates. This means that any tensors created within the function are not accessible outside of it.  In your case, you're trying to access the output of a layer from a model (gen1) that was likely created within a tf.function. This is not allowed as per TensorFlow's execution model.  To resolve this issue, you need to ensure that the model gen1 is created outside of a tf.function context. If the model is being created within a tf.function decorated function, you should move the model creation outside of this function.  If you don't have control over where and how gen1 is created, an alternative solution would be to create a new model with the same architecture as gen1 but created outside of a tf.function context. You can then load the weights from gen1 into this new model and proceed with your operation.

# Assuming the generator is a model created within a tf.function context

In [ ]:
def create_generator(dropout_rate, dropout_type='bernoulli'):
    gen_config = {
            'drop_rate': dropout_rate,
            # 'dropout_type': 'bernoulli_structured',
            'dropout_type': dropout_type,
            # 'drop_kwargs': {'patch_size': 3}
    }
    generator = RichMCDropFunc(**gen_config)
    generator.build((None, 3))
    return generator

In [ ]:
# Create a new model with the same architecture
new_gen = create_generator(0.1)
new_gen.set_weights(gen1.get_weights())

# Now you can create a new model that outputs from the 15th layer
input_layer = new_gen.input
output_layer = new_gen.layers[14].output  # Index is 14 because layer indexing starts from 0

# Create a new model
new_model = Model(input_layer, output_layer)

# Now you can use `new_model` to predict on your data
embedding_output = new_model.predict(input_data)

embedding_output

Layer 0
Layer 1
Layer 2
Layer 3
Layer 4
1/1 [==============================] - 0s 139ms/step


array([[ 0.1970878 , -0.24459776,  1.4919145 , -0.45148534, -0.08470431,
         0.5635197 , -0.97210366, -1.3280557 ,  0.03235679,  0.44038948,
         0.25111318, -0.95786685,  0.07753997,  0.84519356,  0.7116152 ,
         0.12451969,  0.36327466,  0.2476968 ,  0.66545844,  0.59294915,
         0.20050749, -0.19333921, -1.1475161 , -0.36302882, -0.20809755,
         0.27293456,  0.11388439, -0.05502814, -0.6293955 ,  0.3655545 ,
        -0.24131197, -0.70723915,  0.81003416,  0.01305694,  0.23496412,
         0.11750372,  0.48235074, -0.71726525, -0.23201275, -0.3864569 ,
        -0.610474  ,  0.04682522,  0.63666946,  0.11976689, -0.15143754,
         0.8800062 , -1.1031467 ,  0.71379155, -0.1876362 , -0.6030136 ,
        -1.0119    , -0.8003013 , -1.6790619 ,  0.15865615, -0.43839175,
        -0.38478366, -0.9927331 , -0.36635554,  0.14383674, -0.4745398 ,
        -0.06113171,  0.44382176,  1.4804621 ,  0.3410128 , -0.63060015,
        -0.17406036,  0.56266207,  0.23368126, -0.0